# Imports

In [59]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,accuracy_score
from sklearn.preprocessing import MinMaxScaler

# Data reading and preparation

In [82]:
data = pd.read_csv('am_Train.csv')
label = pd.read_csv("am_StepOne.csv")
test = pd.read_csv("InputTest.csv")
data = data.set_index('Index')
label = label.set_index('Index')
test = test.set_index('Index')
data

,House_id,TimeStep_0,TimeStep_1,TimeStep_2,TimeStep_3,TimeStep_4,TimeStep_5,TimeStep_6,TimeStep_7,TimeStep_8,...,TimeStep_2150,TimeStep_2151,TimeStep_2152,TimeStep_2153,TimeStep_2154,TimeStep_2155,TimeStep_2156,TimeStep_2157,TimeStep_2158,TimeStep_2159
Index,,,,,,,,,,,,,,,,,,,,,
1,1,180.000000,180.0,180.0,181.0,180.0,180.0,180.000000,181.0,180.0,...,238.0,238.0,238.0,236.0,238.333333,238.333333,236.0,234.0,234.0,235.0
2,1,2437.000000,2426.0,2148.0,645.0,642.0,642.0,2256.666667,2436.0,2436.0,...,235.0,233.0,233.0,236.0,236.000000,236.000000,234.0,234.0,230.0,233.0
3,1,232.000000,232.0,232.5,233.0,233.0,234.0,233.000000,233.0,232.0,...,185.0,182.0,181.0,180.0,182.000000,181.500000,181.0,180.0,180.0,180.0
4,1,180.333333,181.0,180.0,184.0,181.0,180.0,183.000000,184.0,184.0,...,357.0,351.0,349.0,351.0,358.000000,349.000000,348.5,340.0,346.0,344.0
5,1,344.000000,341.0,341.0,327.0,327.0,318.0,318.000000,313.0,313.0,...,235.0,238.0,238.0,235.0,235.000000,235.000000,233.0,233.0,235.0,233.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15711,16,242.000000,241.0,242.5,243.0,242.5,244.0,243.000000,240.0,237.5,...,566.0,787.0,532.0,739.0,370.000000,919.000000,599.5,863.0,902.0,938.0
15712,16,362.000000,847.0,568.0,843.0,208.0,862.0,241.000000,845.0,573.0,...,177.0,176.5,177.0,176.0,177.000000,177.000000,178.0,178.0,177.0,178.0
15713,16,355.500000,358.0,361.0,348.0,369.0,381.0,380.500000,380.0,351.5,...,240.5,240.0,240.0,241.0,239.500000,240.000000,240.0,240.0,241.0,239.0


In [84]:
X, y = data.drop('House_id', axis=1), label.drop('House_id', axis=1)
testx = test.drop("House_id",axis=1)

In [85]:
X.insert(len(X.columns), "min", data.drop("House_id",axis=1).min(axis=1))
X.insert(len(X.columns), "max", data.drop("House_id",axis=1).max(axis=1))
X.insert(len(X.columns), "std", data.drop("House_id",axis=1).std(axis=1))
X.insert(len(X.columns), "var", data.drop("House_id",axis=1).var(axis=1))
X.insert(len(X.columns), "mean", data.drop("House_id",axis=1).mean(axis=1))
X

,TimeStep_0,TimeStep_1,TimeStep_2,TimeStep_3,TimeStep_4,TimeStep_5,TimeStep_6,TimeStep_7,TimeStep_8,TimeStep_9,...,TimeStep_2155,TimeStep_2156,TimeStep_2157,TimeStep_2158,TimeStep_2159,min,max,std,var,mean
Index,,,,,,,,,,,,,,,,,,,,,
1,180.000000,180.0,180.0,181.0,180.0,180.0,180.000000,181.0,180.0,181.0,...,238.333333,236.0,234.0,234.0,235.0,179.000000,3566.0,346.031985,119738.134353,274.770139
2,2437.000000,2426.0,2148.0,645.0,642.0,642.0,2256.666667,2436.0,2436.0,2212.0,...,236.000000,234.0,234.0,230.0,233.0,180.000000,5444.0,517.594892,267904.472169,464.613580
3,232.000000,232.0,232.5,233.0,233.0,234.0,233.000000,233.0,232.0,234.0,...,181.500000,181.0,180.0,180.0,180.0,178.666667,553.0,52.104176,2714.845137,212.391667
4,180.333333,181.0,180.0,184.0,181.0,180.0,183.000000,184.0,184.0,183.0,...,349.000000,348.5,340.0,346.0,344.0,151.000000,2701.0,267.951248,71797.871555,302.598225
5,344.000000,341.0,341.0,327.0,327.0,318.0,318.000000,313.0,313.0,315.0,...,235.000000,233.0,233.0,235.0,233.0,153.000000,399.0,37.034982,1371.589907,223.516049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15711,242.000000,241.0,242.5,243.0,242.5,244.0,243.000000,240.0,237.5,239.0,...,919.000000,599.5,863.0,902.0,938.0,173.000000,5155.0,866.008964,749971.526469,566.527238
15712,362.000000,847.0,568.0,843.0,208.0,862.0,241.000000,845.0,573.0,797.0,...,177.000000,178.0,178.0,177.0,178.0,145.000000,1534.0,148.193970,21961.452701,250.381481
15713,355.500000,358.0,361.0,348.0,369.0,381.0,380.500000,380.0,351.5,380.0,...,240.000000,240.0,240.0,241.0,239.0,176.000000,3237.0,546.311174,298455.898765,439.512114


In [86]:
testx.insert(len(testx.columns), "min", test.drop("House_id",axis=1).min(axis=1))
testx.insert(len(testx.columns), "max", test.drop("House_id",axis=1).max(axis=1))
testx.insert(len(testx.columns), "std", test.drop("House_id",axis=1).std(axis=1))
testx.insert(len(testx.columns), "var", test.drop("House_id",axis=1).var(axis=1))
testx.insert(len(testx.columns), "mean", test.drop("House_id",axis=1).mean(axis=1))
testx

,TimeStep_0,TimeStep_1,TimeStep_2,TimeStep_3,TimeStep_4,TimeStep_5,TimeStep_6,TimeStep_7,TimeStep_8,TimeStep_9,...,TimeStep_2155,TimeStep_2156,TimeStep_2157,TimeStep_2158,TimeStep_2159,min,max,std,var,mean
Index,,,,,,,,,,,,,,,,,,,,,
0,595.000000,595.0,601.0,592.0,592.0,581.0,592.000000,592.000000,579.0,584.5,...,2541.0,2653.000000,2671.0,2671.0,2652.0,117.0,3696.5,451.849066,204167.578807,334.352238
1,2682.666667,2622.0,2629.0,2635.0,2624.0,2635.0,2636.333333,2652.000000,2652.0,2635.5,...,119.5,119.500000,121.0,121.0,121.0,117.0,3506.0,564.755083,318948.304127,497.435648
2,529.000000,534.0,540.0,530.5,527.0,527.0,533.333333,528.000000,528.0,527.0,...,200.0,201.000000,202.0,202.0,198.0,121.0,5802.0,872.159038,760661.387255,869.328318
3,197.500000,197.5,198.0,200.0,200.0,213.0,207.333333,207.000000,206.0,206.0,...,269.0,264.333333,267.0,267.0,272.0,117.0,3399.0,332.342792,110451.731513,303.780401
4,291.500000,285.0,283.0,284.0,284.0,283.0,280.333333,280.333333,278.0,285.0,...,121.0,120.000000,120.0,120.0,121.0,117.0,3624.0,390.979041,152864.610879,421.444907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2483,1789.000000,1789.0,1789.0,1789.0,1789.0,1789.0,1789.000000,1789.000000,1789.0,1789.0,...,1789.0,1789.000000,1789.0,1789.0,1789.0,1789.0,1789.0,0.000000,0.000000,1789.000000
2484,1789.000000,1789.0,1789.0,1789.0,1789.0,1789.0,1789.000000,1789.000000,1789.0,1789.0,...,1789.0,1789.000000,1789.0,1789.0,1789.0,1789.0,1789.0,0.000000,0.000000,1789.000000
2485,1789.000000,1789.0,1789.0,1789.0,1789.0,1789.0,1789.000000,1789.000000,1789.0,1789.0,...,1789.0,1789.000000,1789.0,1789.0,1789.0,1789.0,1789.0,0.000000,0.000000,1789.000000


In [87]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

# XGBoost Algorithm

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [91]:
dtrain_reg = xgb.DMatrix(X_train, y_train)

dtest_reg = xgb.DMatrix(X_test, y_test)

In [92]:
test_X = xgb.DMatrix(scaler.transform(testx))

In [104]:
params = {"alpha":2,"subsample":0.829,'min_child_weight':12,'gamma':0.1,'max_depth': 4,'eta':0.1,"objective": "reg:squarederror", "tree_method": "gpu_hist"}
evalist = [(dtrain_reg, 'train'), (dtest_reg, 'eval')]
n = 1000
bst = xgb.train(params, dtrain_reg, n, evalist,early_stopping_rounds=10)

[0]	train-rmse:0.47313	eval-rmse:0.47324
[1]	train-rmse:0.44995	eval-rmse:0.45022
[2]	train-rmse:0.43004	eval-rmse:0.43058
[3]	train-rmse:0.41297	eval-rmse:0.41381
[4]	train-rmse:0.39826	eval-rmse:0.39937
[5]	train-rmse:0.38584	eval-rmse:0.38735
[6]	train-rmse:0.37526	eval-rmse:0.37714
[7]	train-rmse:0.36623	eval-rmse:0.36858
[8]	train-rmse:0.35854	eval-rmse:0.36134
[9]	train-rmse:0.35151	eval-rmse:0.35495
[10]	train-rmse:0.34589	eval-rmse:0.34965
[11]	train-rmse:0.34093	eval-rmse:0.34508
[12]	train-rmse:0.33652	eval-rmse:0.34113
[13]	train-rmse:0.33275	eval-rmse:0.33796
[14]	train-rmse:0.32935	eval-rmse:0.33506
[15]	train-rmse:0.32633	eval-rmse:0.33253
[16]	train-rmse:0.32347	eval-rmse:0.33014
[17]	train-rmse:0.32074	eval-rmse:0.32795
[18]	train-rmse:0.31825	eval-rmse:0.32603
[19]	train-rmse:0.31612	eval-rmse:0.32450
[20]	train-rmse:0.31411	eval-rmse:0.32304
[21]	train-rmse:0.31208	eval-rmse:0.32168
[22]	train-rmse:0.30999	eval-rmse:0.32016
[23]	train-rmse:0.30843	eval-rmse:0.31909
[2

[194]	train-rmse:0.19803	eval-rmse:0.25905
[195]	train-rmse:0.19770	eval-rmse:0.25889
[196]	train-rmse:0.19734	eval-rmse:0.25872
[197]	train-rmse:0.19687	eval-rmse:0.25847
[198]	train-rmse:0.19650	eval-rmse:0.25828
[199]	train-rmse:0.19610	eval-rmse:0.25805
[200]	train-rmse:0.19570	eval-rmse:0.25785
[201]	train-rmse:0.19530	eval-rmse:0.25769
[202]	train-rmse:0.19489	eval-rmse:0.25745
[203]	train-rmse:0.19448	eval-rmse:0.25721
[204]	train-rmse:0.19413	eval-rmse:0.25705
[205]	train-rmse:0.19373	eval-rmse:0.25685
[206]	train-rmse:0.19341	eval-rmse:0.25669
[207]	train-rmse:0.19310	eval-rmse:0.25653
[208]	train-rmse:0.19281	eval-rmse:0.25642
[209]	train-rmse:0.19244	eval-rmse:0.25624
[210]	train-rmse:0.19209	eval-rmse:0.25599
[211]	train-rmse:0.19180	eval-rmse:0.25580
[212]	train-rmse:0.19147	eval-rmse:0.25565
[213]	train-rmse:0.19106	eval-rmse:0.25542
[214]	train-rmse:0.19076	eval-rmse:0.25525
[215]	train-rmse:0.19038	eval-rmse:0.25505
[216]	train-rmse:0.19000	eval-rmse:0.25487
[217]	train

[385]	train-rmse:0.14678	eval-rmse:0.23261
[386]	train-rmse:0.14664	eval-rmse:0.23255
[387]	train-rmse:0.14648	eval-rmse:0.23245
[388]	train-rmse:0.14627	eval-rmse:0.23238
[389]	train-rmse:0.14613	eval-rmse:0.23230
[390]	train-rmse:0.14595	eval-rmse:0.23218
[391]	train-rmse:0.14585	eval-rmse:0.23213
[392]	train-rmse:0.14565	eval-rmse:0.23206
[393]	train-rmse:0.14553	eval-rmse:0.23199
[394]	train-rmse:0.14534	eval-rmse:0.23190
[395]	train-rmse:0.14522	eval-rmse:0.23183
[396]	train-rmse:0.14508	eval-rmse:0.23177
[397]	train-rmse:0.14491	eval-rmse:0.23166
[398]	train-rmse:0.14473	eval-rmse:0.23155
[399]	train-rmse:0.14451	eval-rmse:0.23146
[400]	train-rmse:0.14436	eval-rmse:0.23138
[401]	train-rmse:0.14422	eval-rmse:0.23134
[402]	train-rmse:0.14407	eval-rmse:0.23126
[403]	train-rmse:0.14394	eval-rmse:0.23120
[404]	train-rmse:0.14383	eval-rmse:0.23114
[405]	train-rmse:0.14362	eval-rmse:0.23104
[406]	train-rmse:0.14342	eval-rmse:0.23093
[407]	train-rmse:0.14329	eval-rmse:0.23085
[408]	train

[766]	train-rmse:0.12699	eval-rmse:0.22240
[767]	train-rmse:0.12699	eval-rmse:0.22240
[768]	train-rmse:0.12691	eval-rmse:0.22235
[769]	train-rmse:0.12686	eval-rmse:0.22234
[770]	train-rmse:0.12681	eval-rmse:0.22233
[771]	train-rmse:0.12681	eval-rmse:0.22233
[772]	train-rmse:0.12681	eval-rmse:0.22233
[773]	train-rmse:0.12681	eval-rmse:0.22233
[774]	train-rmse:0.12681	eval-rmse:0.22233
[775]	train-rmse:0.12681	eval-rmse:0.22233
[776]	train-rmse:0.12681	eval-rmse:0.22233
[777]	train-rmse:0.12681	eval-rmse:0.22233
[778]	train-rmse:0.12681	eval-rmse:0.22233
[779]	train-rmse:0.12681	eval-rmse:0.22233
[780]	train-rmse:0.12681	eval-rmse:0.22233


In [94]:
bst.save_model("best_model")

# Prediction and Results

In [98]:
test_y =bst.predict(test_X)

In [53]:
results = test_y

In [100]:
inputes = pd.read_csv('InputTrain.csv')

In [101]:
output_res = pd.DataFrame(test_y, columns=["Washing Machine", "Dishwasher", "Tumble Dryer", "Microwave", "Kettle"])
output_res.insert(0, "Index", inputes["Index"])
output_res.to_csv("./Results_test.csv", index=False)